In [1]:
import pandas as pd
import numpy as np
import datetime as dt
from scipy.optimize import *

In [2]:
#DATA LOADING

orders = pd.read_excel('orders_test2f.xlsx')
products=pd.read_excel('orders_test2f.xlsx',sheet_name  = 1)
lines=pd.read_excel('orders_test2f.xlsx',sheet_name  = 2)

In [3]:
#GLOBAL VARIABLES

#Start planning day
startplan=dt.datetime(2022,3,10)
orders['startplan']=[startplan if orders.MDDate[i]<pd.Timestamp(startplan) else orders.MDDate[i] for i in orders.index]

#Planned efficiency
Eff=0.75

#Number of lines
nlines=lines.shape[0]
line=dict(zip(np.linspace(1,nlines,nlines).astype(int),lines.Line))
linecap=dict(zip(np.linspace(1,nlines,nlines).astype(int),lines.Capacity))

#Number of CO's
nCOs=orders.shape[0]
CO=orders.CO

#Calculate order shipment day
orders['ShipDay']=(orders.ReqShipDate.dt.date-orders.startplan.dt.date).dt.days

#Array: order ship days
order_shipdates=orders.ShipDay.values
#2D array: 1 if shipdate[i]<=shipdate[j], 0 otherwise
shipdates_mask = (order_shipdates*np.ones((nCOs,nCOs))) <= (order_shipdates*np.ones((nCOs,nCOs))).T
# 2D array: 0 if same style, 10000000 otherwise
# items_mask=np.logical_not(np.tile(orders.Item.values,(10,1))==np.tile(orders.Item.values,(10,1)).T)*10000000


#Dictionary: product standard minutes
product_stdmins=dict(zip(products.Product,products.SMV))

#Calculate order loading minutes
orders['Load']=orders.Item.map(product_stdmins)*orders.Qty

#Array: order loading minutes
order_loadmins=orders.Load.values

#Array: line daily capacity
line_capacitymins=np.array([lines.Capacity.values*order_shipdates[i]*Eff for i in range(nCOs)])


In [4]:
#BUILD OPTIMIZATION PROBLEM

#Create decision variable
ranges=(slice(1,nlines+1,1),)*nCOs

#Create vectorized loss function
def Loss(x):
    x=x.astype(int)
    AccCap=line_capacitymins[:,0]
    AccLoad=(((x*np.ones((nCOs,nCOs)))==(x*np.ones((nCOs,nCOs))).T)*shipdates_mask*order_loadmins).sum(axis=1)#*items_mask
    Loss=np.sum(AccLoad-AccCap)

    #Add 2h of loss for each MH change in the lines(assuming the start of planning MH is different)
    nChng_mask=len(set([i+j for i, j in zip(orders.Item.values.astype(str), x.astype(str))]))*120
    Loss+=nChng_mask
    return Loss

#Solve the problem
start_time = dt.datetime.now()
result=brute(Loss,ranges,disp=True).astype(int)
end_time = dt.datetime.now()
print('Duration: {}'.format(end_time - start_time))

Optimization terminated successfully.
         Current function value: 106580.000000
         Iterations: 12
         Function evaluations: 143
Duration: 0:00:02.851374


In [5]:
#Calculate start and end production dates
def finalLoss(x):
    AccCap=line_capacitymins[:,0]
    AccLoad=(((x*np.ones((nCOs,nCOs)))==(x*np.ones((nCOs,nCOs))).T)*shipdates_mask*order_loadmins).sum(axis=1)

    #deduct from AccCap only if there is change of MH (assuming the start of planning MH is different)
    chng_df=pd.DataFrame({'Item':orders.Item.values.astype(str),'LineNum':x.astype(str),'AccCap':AccCap})
    chng_df.sort_values(by=['LineNum','AccCap'],axis=0,inplace=True)
    chng_df.reset_index(inplace=True)

    chng_df['Item+Line']=chng_df['Item'] + chng_df['LineNum']
    nChng_mask= np.tile(chng_df['Item+Line'],(10,1)) == np.tile(chng_df['Item+Line'],(10,1)).T
    nChng_mask1=nChng_mask.cumsum(axis=1) *nChng_mask
    nChng_mask1=(nChng_mask1.max(axis=0)==1)

    chng_df['AccCap']=chng_df['AccCap']-(nChng_mask1*120)

    chng_df.sort_values(by=['index'],axis=0,inplace=True)
    chng_df.drop(columns='index',axis=1,inplace=True)
    chng_df.reset_index(inplace=True,drop=True)

    return chng_df['AccCap'], AccLoad

pplan=orders.copy(deep=True)
pplan['AccCap'],pplan['AccLoad']=finalLoss(result)
pplan['LineNum']=result
pplan['Line']=pplan.LineNum.map(line)
pplan.sort_values(by=['LineNum','AccLoad'],axis=0,inplace=True)
pplan.reset_index(inplace=True,drop=True)
pplan['ProdDays']=pplan.Load.values/pplan.LineNum.map(linecap) #I divided by eff from
pplan['StartProd']=pplan.startplan


for i in range(nCOs):
    if (i>0):
        if pplan.LineNum[i]==pplan.LineNum[i-1]:pplan.loc[i,'StartProd']=pplan.EndProd[i-1]
    pplan.loc[i,'EndProd']=pplan.StartProd[i]+dt.timedelta(pplan.ProdDays[i])


pplan['OTD']=(pplan.EndProd<=pplan.ReqShipDate)*1
pplan['DelayDays']=(pplan.EndProd.dt.date-pplan.ReqShipDate.dt.date).dt.days

pplan

,CO,Item,Qty,MDDate,ReqShipDate,startplan,ShipDay,Load,AccCap,AccLoad,LineNum,Line,ProdDays,StartProd,EndProd,OTD,DelayDays
0,VDV001,G0991,10000,2022-03-19,2022-03-29,2022-03-19,10,15000.0,28230.0,15000.0,1,ML001,3.968254,2022-03-19 00:00:00.000000,2022-03-22 23:14:17.142857,1,-7
1,MAS002,G2122,30000,2022-03-08,2022-04-08,2022-03-10,29,33000.0,82095.0,48000.0,1,ML001,8.730159,2022-03-22 23:14:17.142857,2022-03-31 16:45:42.857143,1,-8
2,MAS001,G2121,50000,2022-02-11,2022-04-17,2022-03-10,38,60000.0,107610.0,108000.0,1,ML001,15.873016,2022-03-31 16:45:42.857143,2022-04-16 13:42:51.428572,1,-1
3,TFB002,G2345,215000,2022-02-03,2022-05-23,2022-03-10,74,161250.0,209670.0,269250.0,1,ML001,42.658730,2022-04-16 13:42:51.428572,2022-05-29 05:31:25.714286,0,6
4,TFB003,G2815,40000,2022-03-07,2022-04-07,2022-03-10,28,32000.0,79260.0,32000.0,2,ML002,8.465608,2022-03-10 00:00:00.000000,2022-03-18 11:10:28.571429,1,-20
5,TFB001,G2815,100000,2022-03-14,2022-04-24,2022-03-14,41,80000.0,116235.0,112000.0,2,ML002,21.164021,2022-03-18 11:10:28.571429,2022-04-08 15:06:40.000000,1,-16
6,GAP001,G2111,300000,2022-03-19,2022-05-19,2022-03-19,61,150000.0,172815.0,262000.0,2,ML002,39.682540,2022-04-08 15:06:40.000000,2022-05-18 07:29:31.428571,1,-1
7,TFB004,G2815,30000,2022-03-05,2022-04-05,2022-03-10,26,24000.0,73590.0,24000.0,3,ML003,6.349206,2022-03-10 00:00:00.000000,2022-03-16 08:22:51.428571,1,-20
8,MAS003,G2121,120000,2022-03-12,2022-05-12,2022-03-12,61,144000.0,172815.0,168000.0,3,ML003,38.095238,2022-03-16 08:22:51.428571,2022-04-23 10:40:00.000000,1,-19
9,GAP002,G2111,350000,2022-01-31,2022-05-31,2022-03-10,82,175000.0,232350.0,343000.0,3,ML003,46.296296,2022-04-23 10:40:00.000000,2022-06-08 17:46:40.000000,0,8


In [6]:
#Main results
print('OTIF: {:,.0%}'.format(pplan.OTD.sum()/pplan.shape[0]))
print('Total Delay Days: {:,.0f}'.format(pplan.loc[pplan.DelayDays>0,'DelayDays'].sum()))
# print('Capacity shortage in mins (MH change not included): {:,.0f}'.format((pplan.AccLoad-pplan.AccCap).sum()))
print('MH change minutes: {:,.0f}'.format(len(set([i+j for i, j in zip(orders.Item.values.astype(str), result.astype(str))]))*120))

OTIF: 80%
Total Delay Days: 14
MH change minutes: 1,080
